In [0]:
# To allow Collabs to access Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# To Access Kaggle
!pip install kaggle

from google.colab import files
files.upload() 

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d moltean/fruits

In [0]:
import zipfile
local_zip = '/content/fruits.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [0]:
import tensorflow 
from math import ceil
from tensorflow.keras import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten


In [0]:
isTrain = 0
useTrainedModel = 1 - isTrain
nEpochs = 5
batchSizeTrain = 32
batchSizeTest = 32

In [0]:
if(isTrain):
    TrainDataGenerator = ImageDataGenerator()
    TrainGenerator = TrainDataGenerator.flow_from_directory(
        "./fruits-360/Training/",
        target_size=(100, 100),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batchSizeTrain,
        shuffle=True
    )

In [0]:
if(isTrain):
    model = Sequential()
    model.add(Conv2D(32, (3,3), strides=1, padding='same', input_shape=TrainGenerator.image_shape ))
    model.add(Activation('tanh'))
    model.add(Conv2D(64, (3,3), strides=1, padding='valid'))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3,3), strides=1, padding='valid'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(TrainGenerator.num_classes))
    model.add(Activation('softmax'))

    opt = tensorflow.keras.optimizers.SGD(lr=0.001)
    
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    cb = tensorflow.keras.callbacks.EarlyStopping(
        monitor='acc',
        min_delta=0.001,
        patience=0,
        verbose=0,
        mode='max'
    )

In [0]:
if(isTrain):
    #TODO: Create this directory in your drive "/Machine_Learning/Models_Training/"
    index = 0
    while True:
        res = model.fit_generator(
                TrainGenerator,
                steps_per_epoch = ceil(TrainGenerator.classes.size / TrainGenerator.batch_size),
                epochs = nEpochs,
                callbacks = [cb]
            )
        model.save('drive/Machine_Learning/Models_Training/' + str(index) +'KaggleFruits.h5')
        del model
        if len(res.epoch) != nEpochs:
            break

        model = load_model('drive/Machine_Learning/Models_Training/' + str(index) +'KaggleFruits.h5')
        index += 1
        index = index % 2

In [0]:
if(useTrainedModel):
    #TODO: download this File from the my Google Drive (https://drive.google.com/file/d/1a9u_wEviYj3DJJpcTqiBhdsJTLVLo7AF/view?usp=sharing)
    #      and move it to your drive in this directory
    model = load_model('drive/Machine_Learning/TrainedModels/KaggleFruits.h5')

In [21]:
TestDataGenerator = ImageDataGenerator()
TestGenerator = TestDataGenerator.flow_from_directory(
    './fruits-360/Test/',
    target_size=(100, 100),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batchSizeTrain,
    shuffle=True
)

Found 16421 images belonging to 95 classes.


In [22]:
scores = model.evaluate_generator(TestGenerator, steps=ceil(TestGenerator.classes.size / TestGenerator.batch_size))
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Test loss: 0.12384282786703527
Test accuracy: 0.9598685
